In [ ]:
import torch.nn as nn
import torch

from torch import nn
from functools import partial

In [ ]:
def pair(val):
    return (val, val) if not isinstance(val, tuple) else val


In [ ]:
def check_sizes(image_size, patch_size):
    image_height, image_width = pair(image_size)
    patch_height, patch_width = pair(patch_size)
    assert (image_height % patch_height) == 0 and (
                image_width % patch_width) == 0, 'image height and width must be divisible by patch size'
    num_patches = (image_height // patch_height) * (image_width // patch_width)
    return num_patches

In [ ]:
class PreNormResidual(nn.Module):
    def __init__(self, dim, fn):
        super().__init__()
        self.fn = fn
        self.norm = nn.LayerNorm(dim)

    def forward(self, x):
        return self.fn(self.norm(x)) + x

In [ ]:
class FeedForward(nn.Module):
    def __init__(self, dim, hidden_dim, dropout=0., dense=nn.Linear):
        super().__init__()
        self.net = nn.Sequential(
            dense(dim, hidden_dim),
            nn.GELU(),
            nn.Dropout(dropout),
            dense(hidden_dim, dim),
            nn.Dropout(dropout)
        )

    def forward(self, x):
        x = self.net(x)
        return self.net(x)


In [ ]:
class MLPMixer(nn.Module):
    def __init__(self, num_patches, d_model, depth, expansion_factor=4, dropout=0.):
        super().__init__()

        chan_first, chan_last = partial(nn.Conv1d, kernel_size=1), nn.Linear
        self.model = nn.Sequential(
            *[nn.Sequential(
                PreNormResidual(d_model, FeedForward(num_patches, num_patches * expansion_factor, dropout, chan_first)),
                PreNormResidual(d_model, FeedForward(d_model, d_model * expansion_factor, dropout, chan_last))
            ) for _ in range(depth)]
        )

    def forward(self, x):
        return self.model(x)

In [ ]:
class MLPMixerForImageClassification(MLPMixer):
    def __init__(
            self,
            in_channels=256,
            d_model=512,
            num_classes=1000,
            patch_size=4,
            image_size=56,
            depth=4,
            expansion_factor=4):
        num_patches = check_sizes(image_size, patch_size)

        super().__init__(num_patches, d_model, depth, expansion_factor)

        self.patcher = nn.Sequential(
            nn.Conv2d(in_channels, d_model, kernel_size=patch_size, stride=patch_size),  # 每个token是512维
        )

        self.active = nn.LayerNorm(d_model)
        # self.mlp_head = nn.Sequential(
        #     nn.Linear(d_model, num_classes)
        # )

    def forward(self, x):


        patches = self.patcher(x)

        batch_size, num_channels, _, _ = patches.shape  # 32， 512， 14， 14
        patches = patches.permute(0, 2, 3, 1)  # 32， 14，14， 512
        patches = patches.view(batch_size, -1, num_channels)  # 32, 196, 512
        embedding = self.model(patches)  # 32, 196, 512
        embedding = self.active(embedding)  # 32, 196, 512
        # embedding = embedding.mean(dim=1)                       # 32, 512
        # out = self.mlp_head(embedding)                          # 32, 1000
        embedding = embedding.view(embedding.shape[0], int(embedding.shape[1] ** 0.5), int(embedding.shape[1] ** 0.5),
                                   embedding.shape[2])
        out = embedding.permute(0, 3, 1, 2)
        return out

In [ ]:
class A(MLPMixer):
    def __init__(
            self,
            in_channels=256,  # 3
            d_model=512,  # 512
            num_classes=1000,  # 1000
            patch_size=4,  # 16
            image_size=56,  # 224
            depth=4,  # 12
            expansion_factor=4):  # 4
        num_patches = check_sizes(image_size, patch_size)
        super().__init__(num_patches, d_model, depth, expansion_factor)
        self.mlp0 = MLPMixerForImageClassification(image_size=28)
        self.mlp1 = MLPMixerForImageClassification()
        self.mlp2 = MLPMixerForImageClassification(in_channels=64)

        # self.sp = SplitAttention(d_model)
        #self.fuse = nn.Linear(4*d_model,d_model)
        # self.mlp_head = nn.Sequential(
        #     nn.Linear(d_model, num_classes)
        # )

    def forward(self, x):

        b, c, w, h = x.size()
        x1 = x[:, :, :w // 2, :h // 2]
        x2 = x[:, :, w // 2:, :h // 2]
        x3 = x[:, :, :w // 2, h // 2:]
        x4 = x[:, :, w // 2:, h // 2:]

        x1_out = self.mlp0(x1)
        x2_out = self.mlp0(x2)
        x3_out = self.mlp0(x3)
        x4_out = self.mlp0(x4)
        x5_out = torch.cat((x1_out, x2_out), dim=2)
        x6_out = torch.cat((x3_out, x4_out), dim=2)
        x7_out = torch.cat((x5_out, x6_out), dim=3)

        x8_out = self.mlp1(x)

        x9_out = x[:, ::4, :, :]
        x10_out = x[:, 1::4, :, :]
        x11_out = x[:, 2::4, :, :]
        x12_out = x[:, 3::4, :, :]
        x9_out = self.mlp2(x9_out)
        x10_out = self.mlp2(x10_out)
        x11_out = self.mlp2(x11_out)
        x12_out = self.mlp2(x12_out)

        # out = torch.cat([x9_out, x10_out, x11_out, x12_out], dim=1)
        #out = self.fuse(out.permute(0,2,3,1)).permute(0,3,1,2)

        out = x9_out + x10_out + x11_out + x12_out
        out = out + x7_out + x8_out

        return out

In [ ]:
class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_channel, out_channel, stride=1, downsample=None, **kwargs):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=in_channel, out_channels=out_channel,
                               kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channel)
        self.relu = nn.ReLU()
        self.conv2 = nn.Conv2d(in_channels=out_channel, out_channels=out_channel,
                               kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channel)
        self.downsample = downsample

    def forward(self, x):
        identity = x
        if self.downsample is not None:
            identity = self.downsample(x)

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        out += identity
        out = self.relu(out)

        return out


In [ ]:
class Bottleneck(nn.Module):
    expansion = 4  #

    def __init__(self, in_channel, out_channel, stride=1, downsample=None,
                 groups=1, width_per_group=64):
        super(Bottleneck, self).__init__()

        width = int(out_channel * (width_per_group / 64.)) * groups  #

        self.conv1 = nn.Conv2d(in_channels=in_channel, out_channels=width,
                               kernel_size=1, stride=1, bias=False)  # squeeze channels
        self.bn1 = nn.BatchNorm2d(width)
        # -----------------------------------------
        self.conv2 = nn.Conv2d(in_channels=width, out_channels=width, groups=groups,
                               kernel_size=3, stride=stride, bias=False, padding=1)
        self.bn2 = nn.BatchNorm2d(width)
        # -----------------------------------------
        self.conv3 = nn.Conv2d(in_channels=width, out_channels=out_channel * self.expansion,
                               kernel_size=1, stride=1, bias=False)  # unsqueeze channels
        self.bn3 = nn.BatchNorm2d(out_channel * self.expansion)
        self.relu = nn.ReLU(inplace=True)
        self.downsample = downsample

    def forward(self, x):
        identity = x
        if self.downsample is not None:
            identity = self.downsample(x)

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)

        out = self.conv3(out)
        out = self.bn3(out)

        out += identity
        out = self.relu(out)

        return out

In [ ]:
class ResNet(nn.Module):

    def __init__(self,
                 block,
                 blocks_num,
                 num_classes=1000,
                 include_top=True,
                 groups=1,
                 width_per_group=64):
        super(ResNet, self).__init__()
        self.include_top = include_top
        self.in_channel = 64

        self.groups = groups
        self.width_per_group = width_per_group

        self.conv1 = nn.Conv2d(3, self.in_channel, kernel_size=7, stride=2,
                               padding=3, bias=False)
        self.bn1 = nn.BatchNorm2d(self.in_channel)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)


        self.layer1 = self._make_layer(block, 64, blocks_num[0])
        self.mlpmixer = A(in_channels=256,
                          d_model=512,
                          num_classes=num_classes,
                          patch_size=4,
                          image_size=56,
                          depth=4,
                          expansion_factor=4,
                          # num_patches=check_sizes(image_size=56, patch_size=8)
                          )

        self.layer2 = self._make_layer(block, 128, blocks_num[1], stride=2)
        self.layer3 = self._make_layer(block, 256, blocks_num[2], stride=2)
        self.layer4 = self._make_layer(block, 512, blocks_num[3], stride=2)

        if self.include_top:
            self.avgpool = nn.AdaptiveAvgPool2d((1, 1))  # output size = (1, 1)
            self.fc = nn.Linear(512 * block.expansion, num_classes)


        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')

    def _make_layer(self, block, channel, block_num, stride=1):
        downsample = None
        if stride != 1 or self.in_channel != channel * block.expansion:
            downsample = nn.Sequential(
                nn.Conv2d(self.in_channel, channel * block.expansion, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(channel * block.expansion))

        layers = []
        layers.append(block(self.in_channel,
                            channel,
                            downsample=downsample,
                            stride=stride,
                            groups=self.groups,
                            width_per_group=self.width_per_group))
        self.in_channel = channel * block.expansion

        for _ in range(1, block_num):
            layers.append(block(self.in_channel,
                                channel,
                                groups=self.groups,
                                width_per_group=self.width_per_group))

        return nn.Sequential(*layers)

    def forward(self, x):

        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.mlpmixer(x)
        # x = self.layer2(x)
        # x = self.mlpmixer(x)
        # x = x + y
        x = self.layer3(x)
        x = self.layer4(x)

        if self.include_top:
            x = self.avgpool(x)
            x = torch.flatten(x, 1)
            x = self.fc(x)

        return x


def resnet34(num_classes=1000, include_top=True):
    # https://download.pytorch.org/models/resnet34-333f7ec4.pth
    return ResNet(BasicBlock, [3, 4, 6, 3], num_classes=num_classes, include_top=include_top)


def resnet50(num_classes=1000, include_top=True):
    # https://download.pytorch.org/models/resnet50-19c8e357.pth
    return ResNet(Bottleneck, [3, 4, 6, 3], num_classes=num_classes, include_top=include_top)


def resnet101(num_classes=1000, include_top=True):
    # https://download.pytorch.org/models/resnet101-5d3b4d8f.pth
    return ResNet(Bottleneck, [3, 4, 23, 3], num_classes=num_classes, include_top=include_top)


def resnext50_32x4d(num_classes=1000, include_top=True):
    # https://download.pytorch.org/models/resnext50_32x4d-7cdf4587.pth
    groups = 32
    width_per_group = 4
    return ResNet(Bottleneck, [3, 4, 6, 3],
                  num_classes=num_classes,
                  include_top=include_top,
                  groups=groups,
                  width_per_group=width_per_group)


def resnext101_32x8d(num_classes=1000, include_top=True):
    # https://download.pytorch.org/models/resnext101_32x8d-8ba56ff5.pth
    groups = 32
    width_per_group = 8
    return ResNet(Bottleneck, [3, 4, 23, 3],
                  num_classes=num_classes,
                  include_top=include_top,
                  groups=groups,
                  width_per_group=width_per_group)

